_Aboa Bouadou, Rose Guionnet, Nathan Laîné, Victor Devys, Tino Margely_

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import math
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout
from matplotlib import pyplot as plt
import copy
import time
from pulp import *

In [2]:
SECONDS_PER_DAY = 24*60*60

In [3]:
data = pd.read_excel("data/raw/Orano-données-Double.xlsx", sheet_name="Optim", header=5, usecols=[3,4,5,6,7], nrows=47)

In [4]:
data['disp'].min().to_pydatetime()

datetime.datetime(2021, 1, 1, 0, 0)

In [5]:
def dataframe_to_list(data):
    dataframe = data.copy()
    #dataframe = dataframe
    min_date = dataframe['disp'].min().to_pydatetime()
    dataframe['disp'] = dataframe['disp'].apply(lambda x: (x.to_pydatetime()- min_date).total_seconds()/SECONDS_PER_DAY)
    dataframe['max'] = dataframe['max'].apply(lambda x: (x.to_pydatetime()-min_date).total_seconds()/SECONDS_PER_DAY)
    dataframe['durée'] = dataframe['durée']
    
    return(list(dataframe['disp']),list(dataframe['max']),list(dataframe['durée']))

In [6]:
n = 15
disp, dmax, duree = dataframe_to_list(data)
disp, dmax, duree = disp[:n], dmax[:n], duree[:n]

In [7]:
def solvePLNE(disp, dmax, duree,M,bigM,mu,a):
    
    Tasks=range(len(disp)) #Liste des taches 
    Machine=range(M)
    
    model = LpProblem("Orano", LpMaximize)
    
    # Variables 
    y = LpVariable.dicts("y",((i,m) for i in Tasks for m in Machine),0,1,LpInteger) #y_{i}^m si i est usinée sur la machine m
    x = LpVariable.dicts("x", ((i,j,m) for i in Tasks for j in Tasks for m in Machine),0,1,LpInteger) #x_{i,j}^m=1 si i est usinée avant j
    deb = LpVariable.dicts("deb", Tasks,0, None)
    dem = LpVariable.dicts("dem", Tasks,0, None)
    real = LpVariable.dicts("real", Tasks,0, None)
    marge = LpVariable.dicts("marge", Tasks,0, None)
    z3 = LpVariable.dicts("z3", ((i,j,k,m) for i in Tasks for j in Tasks for k in Tasks for m in Machine),0,1,LpInteger)
    z2 = LpVariable.dicts("z2", ((i,j,m) for i in Tasks for j in Tasks for m in Machine),0,1,LpInteger)
    liv = LpVariable.dicts("liv", Tasks,0, None)
    W = LpVariable("W", lowBound=0)
    
    # Contraintes :
    for j in Tasks:
        
        model += lpSum(y[j,m] for m in Machine)==1
        model += dem[j] >= disp[j]
        model += real[j] == dem[j] + duree[j] 
        model += W <= marge[j]
        model += liv[j] == deb[j] + 30
        model += liv[j] <= dmax[j]
        model += marge[j] == dmax[j] - liv[j]
        model += deb[j] >= real[j]
        
        
        for i in Tasks:
            
            for m in Machine:
                model += dem[j]>= real[i]-bigM*(1-x[i,j,m])
                model += z2[j,i,m] <= y[i,m]
                model += z2[j,i,m] <= y[j,m]
                model += z2[j,i,m] >= (y[i,m]+y[j,m])-1
                model += z2[j,i,m] >= 0
                model += x[j,i,m]+x[j,i,m]==z2[i,j,m]
                model += x[j,i,m]<=z2[i,j,m]
                
            for k in Tasks:
                for m in Machine : 
                    model += z3[i,j,k,m] <= y[i,m]
                    model += z3[i,j,k,m] <= y[j,m]
                    model += z3[i,j,k,m] <= y[k,m]
                    model += z3[i,j,k,m] >= (y[i,m]+y[j,m]+y[k,m])-2
                    model += z3[i,j,k,m] >= 0
                    model += x[i,j,m]+x[j,k,m]-1 <= x[i,k,m]+bigM*(1-z3[i,j,k,m])
                

                    
        
        
        

    
    
    # Objectif : 
    model+= W+ 0.5*lpSum(mu[i]*a[m]*y[i,m] for i in Tasks for m in Machine)
    
    #Resolution du PLNE : 
    starttime=time.time() #Pour avoir le temps d'execution
    model.solve(PULP_CBC_CMD(maxSeconds=15))
    solveTime=time.time()-starttime
    
    #Affichage des éléments de résolution
    f_val=[]
    for i in range(0,len(Tasks)):
        f_val.append(model.variables()[i].varValue)
    order=np.argsort(f_val)
    
        
    print("Temps de résolution = ", solveTime)
    print("Statut de la solution = ", LpStatus[model.status])
    print("Valeur optimale = ", value(model.objective))
    print("Ordre des taches =",order)
    return(solveTime,value(model.objective),list(order))

In [8]:
a=[12,12]
mu=np.random.uniform(0.1,0.7,15)

In [9]:
solvePLNE(disp, dmax, duree,2,10**6,mu,a)

Temps de résolution =  1.4344840049743652
Statut de la solution =  Infeasible
Valeur optimale =  131.70369800623587
Ordre des taches = [ 1  9  7  8 11 14  2  3 10  5  4  0  6 12 13]


(1.4344840049743652,
 131.70369800623587,
 [1, 9, 7, 8, 11, 14, 2, 3, 10, 5, 4, 0, 6, 12, 13])